In [2]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import openai
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import re
import string
from collections import defaultdict
import itertools
from typing import List
from tqdm import tqdm
import os.path
from adjustText import adjust_text
import data
import pickle as pkl

In [3]:
# authors sorted in descending order by num papers
authors_dict_titles = pkl.load(open('../data/authors_dict_titles.pkl', 'rb'))
prompt = 'Here is a list of related machine-learning papers:\n\n> '
authors_save = {}
authors = list(authors_dict_titles.keys())

# restrict to authors that have atleast 3 papers
authors = [a for a in authors if len(authors_dict_titles[a]) > 2]


In [4]:
print(len(authors))

41629


In [7]:
authors_save = pkl.load(open('gen_titles/authors_save_full.pkl', 'rb'))

In [8]:
len(authors_save)

41629

In [9]:
authors_save

{'yoshua bengio': [' Semi-Supervised Reinforcement Learning with Deep Representation Learning\n',
  ' Centralized training of deep neural networks with low communication overhead',
  ' Context-aware politeness: A learning-based approach\n',
  ' Neural Variational Inference and Learning in Boltzmann Machines\n',
  ' Watch Your Step: Guarding Model-Based Reinforcement Learning from Step-wise'],
 'yang liu': [' Models of Explanation for Deep Neural Networks',
  ' Neural Networks for Surrogate Modeling: A Comparative Review\n',
  ' A Study on Generative Adversarial Nets with Wasserstein Distance\n',
  ' On the Universality of Deep Learning\n',
  ' Representation Learning for Time-Series Classification\n'],
 'sergey levine': [' Combining Model-Based and Model-Free Updates in Deep Reinforcement Learning\n',
  ' A Neural Algorithm of Artistic Style',
  ' Quadrupedal Locomotion by Reinforcement Learning',
  ' RLPy: A Multi-platform Toolkit for Reinforcement Learning',
  ' Efficient Exploration